In [235]:
from sklearn.datasets import load_iris
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import pairwise_distances

import cProfile


def getPermutation(totalRange,numberElements):
    random_seed = 10312003
    rng = np.random.RandomState(random_seed)
    permutation = rng.permutation(totalRange)
    return permutation[:numberElements]


def onlineKmeans(X,k=3,b=30,maxiter=1000):
    #print X
    centroids = X[getPermutation(len(X),k)]
    pointsPerClusters = np.zeros([k,1])
    for i in range(maxiter):
        M=X[getPermutation(len(X),b)]
        distances = pairwise_distances(M, centroids, metric='euclidean')
        nearestCenters = np.argmin(distances, axis=1)
        for iter, x in enumerate(M):
            centerIndex = nearestCenters[iter]
            pointsPerClusters[centerIndex] = pointsPerClusters[centerIndex] + 1
            eta = 1/pointsPerClusters[centerIndex]
            centroids[centerIndex] = (1 - eta)*centroids[centerIndex] + eta * x

    return centroids

In [236]:
from timeit import Timer
from functools import partial
from memory_profiler import memory_usage
import timeit

def profile_memory_and_time(function, *args, **kwargs):
    #print "args is ",args
    start_time = timeit.default_timer()
    memory, return_val = memory_usage((function, (args), kwargs), max_usage=True, retval=True)
    elapsed = timeit.default_timer() - start_time
    return memory[0], elapsed,return_val
  

In [237]:
iris = load_iris()
X=iris.data
random_seed = 10312003
rng = np.random.RandomState(random_seed)
permutation = rng.permutation(len(X))
X = X[permutation]
print profile_memory_and_time(onlineKmeans, X, k=2, b=33, maxiter=1000)

(29.66796875, 0.5035429000854492, (array([[ 6.39592343,  2.96967588,  5.06569502,  1.78288449],
       [ 4.88164753,  3.34917624,  1.4443335 ,  0.18176735]]), 10))
